In [91]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import *
from IPython.display import display, HTML
from sklearn.cluster import KMeans

%load_ext sql
%sql sqlite:///followers.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: None@followers.db'

In [236]:
#Map languages to integers
languages = %sql SELECT DISTINCT Language FROM Users
languageMap = {}
i = 0
for row in languages:
    language = str(row).split("\'")[1]
    if language == '':
        continue
    languageMap[language] = i
    i = i+1
print languageMap

Done.
{'el': 23, 'en': 0, 'zh-Hans': 32, 'vi': 14, 'ca': 36, 'en-AU': 39, 'it': 16, 'zh-Hant': 44, 'ar': 4, 'cs': 24, 'id': 12, 'es': 3, 'en-gb': 13, 'ru': 7, 'nl': 25, 'pt': 10, 'no': 37, 'zh-tw': 33, 'nb': 30, 'tr': 19, 'zh-cn': 5, 'th': 1, 'ro': 29, 'pl': 18, 'en-GB': 17, 'fr': 2, 'bg': 42, 'hr': 45, 'de': 6, 'hu': 20, 'fa': 34, 'fi': 28, 'da': 22, 'ja': 9, 'he': 21, 'pt-PT': 31, 'zh-TW': 15, 'sq': 43, 'ko': 8, 'sv': 35, 'es-MX': 40, 'sk': 26, 'zh-CN': 11, 'fil': 41, 'uk': 27, 'sr': 38}


In [250]:
# Create training matrix X from sql tables
numUserFeatures = 7 #The number of columns in the users table (excluding id)
numSamples = 1000
numInfluencers = 12

X = [[0 for x in range(numUserFeatures)] for x in range(numSamples)]
users = %sql SELECT * FROM Users LIMIT :numSamples

for i in range(0,numSamples):
    X[i] = list(users[i])[1:numUserFeatures+1] #ignore the sqlite increasing id field
    userID = X[i][0]
    influencers = %sql SELECT * FROM Influencers WHERE FollowerID = :userID ORDER BY InfluencerName

    if len(influencers) < numInfluencers:  #don't store incomplete data
        X = np.delete(X, i, 0)
        continue
    
    for j in range(0, numInfluencers):
        X[i].append(influencers[j][2])
    
    X[i][5] = languageMap[X[i][5]]

X_printable = DataFrame(X)
X_printable.columns = ['UserID', 'StatusesCount', 'FollowersCount', 'FollowingCount', 'Verified', 'Language', 'UtcOffset', 'BarackObama', 'BillGates', 'Cristiano', 'KimKardashian', 'LilTunechi', 'NASA', 'Oprah', 'Pontifex', 'cnnbrk', 'espn', 'jimmyfallon', 'taylorswift13']
display(X_printable)

X = np.array(X)

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done

IndexError: index 998 is out of bounds for axis 0 with size 998

In [217]:
#Kmeans on two features: StatusesCount and FollowersCount
plt.figure(figsize=(15, 15))
numClusters = 5
randomState = 170
firstFeatureColNo=1
secondFeatureColNo=2

#Extract just the features we care about
X_subset = np.column_stack(([row[firstFeatureColNo] for row in X], [row[secondFeatureColNo] for row in X]))

y_pred = KMeans(n_clusters=numClusters, random_state=randomState).fit_predict(X_subset)

plt.subplot(221)
plt.scatter([row[firstFeatureColNo] for row in X], [row[secondFeatureColNo] for row in X], c=y_pred)
plt.title("Statuses and Followers")
plt.show()
